In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:



# Select the data for analysis
data_selected = data[selected_columns].apply(pd.to_numeric, errors='coerce').dropna()

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data_selected)

# Convert the data to a 3D array for LSTM
def create_dataset(X, time_steps=1):
    Xs = []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
    return np.array(Xs)

TIME_STEPS = 10
X_lstm = create_dataset(X_scaled, TIME_STEPS)

# Split the data into training and testing sets
X_train, X_test = train_test_split(X_lstm, test_size=0.2, random_state=42)

# Build the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(50, input_shape=(TIME_STEPS, X_lstm.shape[2]), return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(50, return_sequences=False))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
history = model_lstm.fit(X_train, X_train[:, -1, 0], epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Plot training history
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('LSTM Training History')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model
model_lstm.evaluate(X_test, X_test[:, -1, 0])
